In [35]:
import cobra
from cobra import Metabolite, Gene, Reaction
from cobra.flux_analysis import single_gene_deletion
import pandas as pd
from medusa.test import load_universal_modelseed
import medusa
from pickle import load
from medusa.flux_analysis import deletion

In [3]:

with open("../results/ensembles/pto_ensemble_100_genes_added_2.pickle", 'rb') as infile:
    ensemble = load(infile)
#universal = load_universal_modelseed()

In [43]:
_24 = ensemble.extract_member("PST_gapfilled_24").summary()
_24.single_gene_deletions

IN FLUXES              OUT FLUXES           OBJECTIVES
---------------------  -------------------  ------------
cpd00105_e    1e+03    cpd00001_e    1e+03  bio1  160
cpd00007_e  816        cpd00011_e    1e+03
cpd00060_e  770        cpd00067_e    1e+03
cpd00281_e  723        cpd00100_e    1e+03
cpd00477_e  591        cpd00162_e    1e+03
cpd00652_e  427        cpd00129_e    1e+03
cpd00082_e  399        cpd00020_e  994
cpd00076_e  341        cpd00280_e  764
cpd00027_e  292        cpd00048_e  657
cpd00322_e   70        cpd00036_e  377
cpd00013_e   58.6      cpd00314_e  176
cpd00009_e   23.5      cpd00268_e   53.3
cpd00118_e   15.3      cpd00149_e    4.58
cpd00254_e   15.3
cpd10516_e    6.1
cpd00249_e    4.42
cpd00035_e    2.29
cpd00107_e    0.567
cpd00051_e    0.564
cpd01017_e    0.548
cpd00099_e    0.495
cpd00264_e    0.248
cpd00132_e    0.161
cpd00034_e    0.0247
cpd00058_e    0.0232
cpd00030_e    0.0177
cpd00063_e    0.00934
cpd00205_e    0.00888


In [31]:
#set media conditions for each ensemble

#complete media has all of the exchange reactions open
complete = {}
for reaction in ensemble.base_model.reactions:
    if "EX_" in reaction.id:
        complete[reaction.id] = 1000

        
#for the other medias, use teh base biolog salts (minus heme) and add the media types needed
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)

gluc_mm = {}
gluc_mm['EX_cpd00027_e'] = 1000
for x in biolog_base_composition.ID:
    gluc_mm["EX_" + x + "_e"] = 1000 

glut_mm = {}
glut_mm['EX_cpd00053_e'] = 1000
for x in biolog_base_composition.ID:
    glut_mm["EX_" + x + "_e"] = 1000 

#load in the anderson metabolites
anderson_df = pd.read_csv("../data/metabolite_ids_Anderson2014.csv")
anderson_media_list = []
met_ids = [met for met in anderson_df.met_id]
for met in met_ids:
    met="EX_" + str(met) + "_e"
    anderson_media_list.append(met)
    
    
anderson = {}
for x in biolog_base_composition.ID:
    anderson["EX_" + x + "_e"] = 1000 
for y in anderson_media_list:
    if y in ensemble.base_model.reactions:
        anderson[y] = 1000
        #print (y + " is IN the model")
    else:
        pass
        #print (y+ " is not in the base model") 

print (complete)        
print ("Glucose : " , gluc_mm)        
print ("Glutamine : " , glut_mm)        
print ("ANDERSON : " , anderson)

{'EX_cpd00794_e': 1000, 'EX_cpd00023_e': 1000, 'EX_cpd00034_e': 1000, 'EX_cpd00041_e': 1000, 'EX_cpd00048_e': 1000, 'EX_cpd00053_e': 1000, 'EX_cpd00080_e': 1000, 'EX_cpd00107_e': 1000, 'EX_cpd00118_e': 1000, 'EX_cpd00119_e': 1000, 'EX_cpd00154_e': 1000, 'EX_cpd00156_e': 1000, 'EX_cpd00210_e': 1000, 'EX_cpd00264_e': 1000, 'EX_cpd00322_e': 1000, 'EX_cpd00637_e': 1000, 'EX_cpd00205_e': 1000, 'EX_cpd00971_e': 1000, 'EX_cpd00067_e': 1000, 'EX_cpd00060_e': 1000, 'EX_cpd00129_e': 1000, 'EX_cpd00027_e': 1000, 'EX_cpd00054_e': 1000, 'EX_cpd00009_e': 1000, 'EX_cpd00001_e': 1000, 'EX_cpd00013_e': 1000, 'EX_cpd00122_e': 1000, 'EX_cpd11596_e': 1000, 'EX_cpd01012_e': 1000, 'EX_cpd00058_e': 1000, 'EX_cpd11593_e': 1000, 'EX_cpd11592_e': 1000, 'EX_cpd11591_e': 1000, 'EX_cpd11590_e': 1000, 'EX_cpd11589_e': 1000, 'EX_cpd11588_e': 1000, 'EX_cpd01017_e': 1000, 'EX_cpd11587_e': 1000, 'EX_cpd11586_e': 1000, 'EX_cpd11585_e': 1000, 'EX_cpd11584_e': 1000, 'EX_cpd11583_e': 1000, 'EX_cpd11582_e': 1000, 'EX_cpd115

In [61]:
ensemble.base_model.medium = complete
model_list = []
for model in ensemble.members:
    model_list.append(model.id)

EX_cpd11493_c does not seem to be an an exchange reaction. Applying bounds anyway.
EX_cpd11416_c does not seem to be an an exchange reaction. Applying bounds anyway.


In [62]:
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [70]:
for key, value in deletion_results.items():
    with pd.ExcelWriter('../data/complete_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
        value.to_excel(writer, sheet_name=key)

ModuleNotFoundError: No module named 'openpyxl'

In [72]:
#for all exchange reactions, use the boundry reaction function in cobrapy, seems more straightforward
#define the media conditions, first the complete media conditions
# to do this set the exchange reactions open for constituents of the complete King's B media
value.to_excel("output.xlsx")

ModuleNotFoundError: No module named 'openpyxl'

In [8]:
lb_medium_dict = {}
for met in lb_medium:
    if ("EX_" + met) in model.exchanges:
        lb_medium_dict["EX_" + met] = 1000
    else:
        print ("no" + met)
        pass
print (lb_medium_dict)


nocpd00018_e
nocpd00033_e
nocpd00039_e
nocpd00046_e
nocpd00065_e
nocpd00066_e
nocpd00069_e
nocpd00084_e
nocpd00091_e
nocpd00126_e
nocpd00161_e
nocpd00182_e
nocpd00184_e
nocpd00215_e
nocpd00218_e
nocpd00220_e
nocpd00226_e
nocpd00239_e
nocpd00311_e
nocpd00383_e
nocpd00393_e
nocpd00438_e
nocpd00541_e
nocpd00644_e
nocpd00654_e
nocpd00793_e
nocpd01048_e
nocpd03424_e
nocpd11595_e
{'EX_cpd00001_e': 1000, 'EX_cpd00007_e': 1000, 'EX_cpd00009_e': 1000, 'EX_cpd00023_e': 1000, 'EX_cpd00027_e': 1000, 'EX_cpd00028_e': 1000, 'EX_cpd00030_e': 1000, 'EX_cpd00034_e': 1000, 'EX_cpd00035_e': 1000, 'EX_cpd00041_e': 1000, 'EX_cpd00048_e': 1000, 'EX_cpd00051_e': 1000, 'EX_cpd00054_e': 1000, 'EX_cpd00058_e': 1000, 'EX_cpd00060_e': 1000, 'EX_cpd00063_e': 1000, 'EX_cpd00067_e': 1000, 'EX_cpd00092_e': 1000, 'EX_cpd00099_e': 1000, 'EX_cpd00107_e': 1000, 'EX_cpd00119_e': 1000, 'EX_cpd00129_e': 1000, 'EX_cpd00149_e': 1000, 'EX_cpd00156_e': 1000, 'EX_cpd00205_e': 1000, 'EX_cpd00246_e': 1000, 'EX_cpd00249_e': 1000, '

In [10]:
#under complete media conditions, run single gene deletions and check for growth
lb_medium_dict['EX_cpd00028_e'] = 0.1
model.medium = lb_medium_dict
#deletion_results_complete_media = single_gene_deletion(model)
#deletion_results_complete_media
model.medium
#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    

{'EX_cpd00023_e': 1000,
 'EX_cpd00034_e': 1000,
 'EX_cpd00041_e': 1000,
 'EX_cpd00048_e': 1000,
 'EX_cpd00107_e': 1000,
 'EX_cpd00119_e': 1000,
 'EX_cpd00156_e': 1000,
 'EX_cpd00322_e': 1000,
 'EX_cpd00205_e': 1000,
 'EX_cpd00971_e': 1000,
 'EX_cpd00067_e': 1000,
 'EX_cpd00060_e': 1000,
 'EX_cpd00129_e': 1000,
 'EX_cpd00027_e': 1000,
 'EX_cpd00054_e': 1000,
 'EX_cpd00009_e': 1000,
 'EX_cpd00001_e': 1000,
 'EX_cpd01012_e': 1000,
 'EX_cpd00058_e': 1000,
 'EX_cpd10516_e': 1000,
 'EX_cpd00531_e': 1000,
 'EX_cpd00149_e': 1000,
 'EX_cpd00035_e': 1000,
 'EX_cpd00051_e': 1000,
 'EX_cpd00249_e': 1000,
 'EX_cpd00007_e': 1000,
 'EX_cpd00030_e': 1000,
 'EX_cpd00063_e': 1000,
 'EX_cpd00099_e': 1000,
 'EX_cpd00254_e': 1000,
 'EX_cpd10515_e': 1000,
 'EX_cpd00028_e': 0.1,
 'EX_cpd00246_e': 1000,
 'EX_cpd00092_e': 1000}

In [11]:
model.summary()

IN FLUXES           OUT FLUXES          OBJECTIVES
------------------  ------------------  --------------
cpd00027_e   1e+03  cpd00082_e  951     bio1  4.53e-14
cpd00092_e  29.1    cpd00011_e   92.4
cpd00067_e   5.25   cpd00107_e   52.9
                    cpd00007_e   30.5
                    cpd00013_e    5.25


In [33]:
deletion_results_complete_media.to_csv("../../../Desktop/single_gene_deletions.csv")

In [43]:
#single gene deltions on the glucose minimal media used for the ensemble flux analysis
model_glucose = cobra.io.read_sbml_model("../results/reconstructions/pst7.xml")
deletion_results_complete_media = single_gene_deletion(model_glucose)
deletion_results_complete_media.to_csv("../results/single_gene_deletions_glucose_mm.csv")

In [74]:
model_glucose.medium

{'EX_cpd00034_e': 1000.0,
 'EX_cpd00048_e': 1000.0,
 'EX_cpd00205_e': 1000.0,
 'EX_cpd00971_e': 1000.0,
 'EX_cpd00067_e': 1000.0,
 'EX_cpd00027_e': 10.0,
 'EX_cpd00009_e': 1000.0,
 'EX_cpd00001_e': 1000.0,
 'EX_cpd00013_e': 1000.0,
 'EX_cpd00058_e': 1000.0,
 'EX_cpd10516_e': 1000.0,
 'EX_cpd00149_e': 1000.0,
 'EX_cpd00007_e': 1000.0,
 'EX_cpd00030_e': 1000.0,
 'EX_cpd00063_e': 1000.0,
 'EX_cpd00099_e': 1000.0,
 'EX_cpd00254_e': 1000.0,
 'EX_cpd10515_e': 1000.0,
 'EX_cpd00028_e': 0.1,
 'EX_cpd00268_e': 1000.0,
 'EX_cpd00092_e': 1000.0}

In [78]:
#define the media conditions, originally from the glucose biolog media, add in the LB media componants and take out the media componants from biolog
#I am using this approach because 
lb_dict = model_glucose.medium.copy()
for met in lb_medium:
    met = "EX_" + met
    if met in model_glucose.medium:
        lb_dict.pop(met)
    lb_dict[met] = 100
lb_dict   

{'EX_cpd00013_e': 1000.0,
 'EX_cpd00268_e': 1000.0,
 'EX_cpd00001_e': 100,
 'EX_cpd00007_e': 100,
 'EX_cpd00009_e': 100,
 'EX_cpd00018_e': 100,
 'EX_cpd00023_e': 100,
 'EX_cpd00027_e': 100,
 'EX_cpd00028_e': 100,
 'EX_cpd00030_e': 100,
 'EX_cpd00033_e': 100,
 'EX_cpd00034_e': 100,
 'EX_cpd00035_e': 100,
 'EX_cpd00039_e': 100,
 'EX_cpd00041_e': 100,
 'EX_cpd00046_e': 100,
 'EX_cpd00048_e': 100,
 'EX_cpd00051_e': 100,
 'EX_cpd00054_e': 100,
 'EX_cpd00058_e': 100,
 'EX_cpd00060_e': 100,
 'EX_cpd00063_e': 100,
 'EX_cpd00065_e': 100,
 'EX_cpd00066_e': 100,
 'EX_cpd00067_e': 100,
 'EX_cpd00069_e': 100,
 'EX_cpd00084_e': 100,
 'EX_cpd00091_e': 100,
 'EX_cpd00092_e': 100,
 'EX_cpd00099_e': 100,
 'EX_cpd00107_e': 100,
 'EX_cpd00119_e': 100,
 'EX_cpd00126_e': 100,
 'EX_cpd00129_e': 100,
 'EX_cpd00149_e': 100,
 'EX_cpd00156_e': 100,
 'EX_cpd00161_e': 100,
 'EX_cpd00182_e': 100,
 'EX_cpd00184_e': 100,
 'EX_cpd00205_e': 100,
 'EX_cpd00215_e': 100,
 'EX_cpd00218_e': 100,
 'EX_cpd00220_e': 100,
 'EX_

In [2]:
model_lb = cobra.io.read_sbml_model("../results/reconstructions/pst7.xml")

NameError: name 'cobra' is not defined

In [ ]:
add_exs = []
add_mets = []
for ex in lb_dict:
    try:
        model_lb.reactions.get_by_id(ex)
    except:
        add_met = universal.metabolites.get_by_id(ex[3:])
        ex_rxn = Reaction(ex)
        ex_rxn.name = "Exchange reaction for " + ex[3:]
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({add_met:-1})
        add_exchanges.append(ex_rxn)
        if add_met.id not in [m.id for m in add_mets]:
            add_mets.append(add_met)
#model_lb.medium = lb_dict

In [87]:
model_lb.add_metabolites(add_mets)
#print ("I have added" + str(add_mets))
for ex in add_exchanges:
    try:
        model_lb.reactions.get_by_id(ex)
    except:
        model_lb.add_reactions(ex)
#print ("I have added" + str(add_exchanges))


TypeError: 'Reaction' object is not iterable

In [66]:
deletion_results_complete_media = single_gene_deletion(model_lb)
deletion_results_complete_media

,growth,status
ids,,
(PSPTO_1287),1.139190e+01,optimal
(PSPTO_5047),3.880012e-15,optimal
(PSPTO_1846),1.614744e+01,optimal
(PSPTO_0364),1.614744e+01,optimal
(PSPTO_0495),1.614744e+01,optimal
(PSPTO_0387),1.139190e+01,optimal
(PSPTO_2055),1.992588e-15,optimal
(PSPTO_3706),1.614744e+01,optimal
(PSPTO_2761),1.614744e+01,optimal


In [78]:
anderson = pd.read_csv("../data/metabolite_ids_Anderson2014.csv")
anderson_media_list = []
met_ids = [met for met in anderson.met_id]
for met in met_ids:
    met="EX_" + met + "_e"
    anderson_media_list.append(met)
    #all the metabolites now have possible exchange reaction title. If the exchange reaction is in the model, it should be opened
    #if it is not in the model, it should not be opened and disregarded bc the model cannot transport it
    #other things were added, (basic biolog salt comp, to ensure growth potental)
print (anderson_media_list)

['EX_cpd17172_e', 'EX_cpd11314_e', 'EX_cpd04453_e', 'EX_cpd04148_e', 'EX_cpd01495_e', 'EX_cpd01293_e', 'EX_cpd01138_e', 'EX_cpd01080_e', 'EX_cpd00762_e', 'EX_cpd00751_e', 'EX_cpd00745_e', 'EX_cpd00386_e', 'EX_cpd00383_e', 'EX_cpd00373_e', 'EX_cpd00368_e', 'EX_cpd00306_e', 'EX_cpd00223_e', 'EX_cpd00222_e', 'EX_cpd00218_e', 'EX_cpd00214_e', 'EX_cpd00161_e', 'EX_cpd00159_e', 'EX_cpd00156_e', 'EX_cpd00153_e', 'EX_cpd00139_e', 'EX_cpd00137_e', 'EX_cpd00136_e', 'EX_cpd00129_e', 'EX_cpd00121_e', 'EX_cpd00108_e', 'EX_cpd00106_e', 'EX_cpd00092_e', 'EX_cpd00085_e', 'EX_cpd00082_e', 'EX_cpd00080_e', 'EX_cpd00076_e', 'EX_cpd00073_e', 'EX_cpd00054_e', 'EX_cpd00041_e', 'EX_cpd00036_e', 'EX_cpd00035_e', 'EX_cpd00033_e', 'EX_cpd00027_e', 'EX_cpd00009_e', 'EX_cpd00001_e', 'EX_cpd00007_e', 'EX_cpd00009_e', 'EX_cpd00011_e', 'EX_cpd00013_e', 'EX_cpd00030_e', 'EX_cpd00034_e', 'EX_cpd00048_e', 'EX_cpd00058_e', 'EX_cpd00063_e', 'EX_cpd00067_e', 'EX_cpd00099_e', 'EX_cpd00149_e', 'EX_cpd00205_e', 'EX_cpd00254_

In [93]:
#close all exchange reactions first, then open up the selecct ones as well as the biomass exchange
for e in model.reactions:
    if "EX_" in e.id:
        e.lower_bound = 0
        e.upper_bound = 0
        #print (e.id, e.bounds)
for ex in model.reactions:
    if ex.id in anderson_media_list:
        ex.lower_bound = -1000
        ex.upper_bound = 1000
        print ("I have opened ", ex.id)
model.reactions.EX_cpd00028_e.lower_bound = -1
model.reactions.EX_cpd11416_c.upper_bound = 1000
model.reactions.EX_cpd11493_c.upper_bound = 1000
model.reactions.EX_cpd00268_e.upper_bound = 1000
model.reactions.EX_cpd00268_e.lower_bound = -1000

I have opened  EX_cpd00034_e
I have opened  EX_cpd00041_e
I have opened  EX_cpd00048_e
I have opened  EX_cpd00080_e
I have opened  EX_cpd00156_e
I have opened  EX_cpd00205_e
I have opened  EX_cpd00971_e
I have opened  EX_cpd00067_e
I have opened  EX_cpd00129_e
I have opened  EX_cpd00027_e
I have opened  EX_cpd00054_e
I have opened  EX_cpd00009_e
I have opened  EX_cpd00001_e
I have opened  EX_cpd00013_e
I have opened  EX_cpd00058_e
I have opened  EX_cpd00137_e
I have opened  EX_cpd10516_e
I have opened  EX_cpd00082_e
I have opened  EX_cpd00106_e
I have opened  EX_cpd00036_e
I have opened  EX_cpd00076_e
I have opened  EX_cpd00149_e
I have opened  EX_cpd00386_e
I have opened  EX_cpd00108_e
I have opened  EX_cpd00222_e
I have opened  EX_cpd00035_e
I have opened  EX_cpd01293_e
I have opened  EX_cpd00121_e
I have opened  EX_cpd00007_e
I have opened  EX_cpd00011_e
I have opened  EX_cpd00030_e
I have opened  EX_cpd00063_e
I have opened  EX_cpd00099_e
I have opened  EX_cpd00254_e
I have opened 

In [94]:
model.summary()

IN FLUXES            OUT FLUXES         OBJECTIVES
-------------------  -----------------  ------------
cpd00106_e    1e+03  cpd00007_e  860    bio1  85.9
cpd00082_e  561      cpd00036_e  856
cpd00013_e  454      cpd00035_e  749
cpd00092_e  240      cpd00011_e  591
cpd00054_e  141      cpd00129_e  251
cpd00027_e   14.6    cpd00067_e  182
cpd00268_e    0.998  cpd00009_e   38.4
cpd00028_e    0.532
cpd10516_e    0.532
cpd00058_e    0.266
cpd00030_e    0.266
cpd00034_e    0.266
cpd00048_e    0.266
cpd00063_e    0.266
cpd00099_e    0.266
cpd00149_e    0.266
cpd00205_e    0.266
cpd00254_e    0.266


In [96]:
deletion_results_anderson_media = single_gene_deletion(model)
deletion_results_anderson_media
deletion_results_anderson_media.to_csv("../results/single_gene_deletions_anderson_media.csv")

In [8]:
len(model.genes)

495